In [1]:
import os
import pandas as pd
import numpy as np
from convert_eprime import convert as ep

In [2]:
source_dir = os.path.join('..','sourcedata')
derivs_dir = os.path.join('..','derivatives','0.3.cleaned')

#### Define a function to clean the N-back data

Stack the blocks vertically instead of horizontally, label the trial rows properly, and tag each trial as a HIT, MISS, FA, CR. We are also ouputting a new CSV data file in the sourcedata folder, all cleaned-up.

In [3]:
def nstack_score_label(fpath,outpath):
    df = pd.read_excel(fpath)
    
    # Hierarchicalize the column index
    df.columns=pd.MultiIndex.from_tuples([
        (df.columns[0].split('.')[0],df.columns[0].split('.')[1]),
        (df.columns[1].split('.')[0],df.columns[1].split('.')[1]),
        (df.columns[2].split('.')[0],df.columns[2].split('.')[1]),
        (df.columns[3].split('.')[0],df.columns[3].split('.')[1]),
        (df.columns[4].split('.')[0],df.columns[4].split('.')[1]),
        (df.columns[5].split('.')[0],df.columns[5].split('.')[1]),
    ])
    
    # Stack blocks, Reset trial row index, and Rename columns to be descriptive
    df = df.stack(0).reset_index().rename(
        columns={'level_0':'trial','level_1':'block'}
    ).sort_values(['block','trial'])
    df['sub'] = os.path.basename(fpath).split('_')[0].split('-')[1]
    df['block'] = df['block'].str[1]
    df['trial'] = df['trial'] + 1
    df = df.set_index(['sub','block','trial'])
    
    # Determine Hits, CRs, FAs
    cr_mask = (df['Rsp'] == 0) & (df['CRsp'] == 0)
    ms_mask = (df['Rsp'] == 0) & (df['CRsp'] == 1)
    fa_mask = (df['Rsp'] == 1) & (df['CRsp'] == 0)
    ht_mask = (df['Rsp'] == 1) & (df['CRsp'] == 1)
    df['CR']   = cr_mask.astype(int)
    df['MISS'] = ms_mask.astype(int)
    df['FA']   = fa_mask.astype(int)
    df['HIT']  = ht_mask.astype(int)
    
    # Convert RT 0 to RT NaN
    df['RT'] = df['RT'].replace(0,np.NaN)
    
    # Output to new CSV datafile
    df.to_csv(outpath)
    print('Output file successfully created- ',outpath)

#### Read all the subject data

Reading only data for the full sample (100-series YA & 200-series OA). Executing N-back data cleaning & EPrime text-to-csv conversion. Setting up for subject-level analysis.

In [4]:
ep_frames=[]
ptb_frames=[]

for s in os.listdir(source_dir):
    if s.startswith('sub-1') or s.startswith('sub-2'):
        sub_dir = os.path.join(source_dir,s)
        for f in os.listdir(sub_dir):
            o = f.split('.')[0]+'.csv'
            fpath = os.path.join(sub_dir,f)
            outpath = os.path.join(sub_dir,o)
            if f.split('_')[-1] == 'beh.txt':
                print(f)
                if os.path.isfile(outpath):
                    print(os.path.basename(outpath),'exists')
                else:
                    ep.text_to_csv(fpath,outpath)
                ep_frames.append(pd.read_csv(outpath))
            if f.split('_')[-1] == 'beh.xlsx':
                print(f)
                if os.path.isfile(outpath):
                    print(os.path.basename(outpath),'exists')
                else:
                    nstack_score_label(fpath,outpath)
                ptb_frames.append(pd.read_csv(outpath))
                ptb_frames[-1]['sub'] = f.split('_')[0].split('-')[1]
print("Done!")

sub-101_task-nback_beh.xlsx
sub-101_task-nback_beh.csv exists
sub-101_task-procspeed_run-1_beh.txt
sub-101_task-procspeed_run-1_beh.csv exists
sub-102_task-nback_beh.xlsx
sub-102_task-nback_beh.csv exists
sub-102_task-procspeed_run-1_beh.txt
sub-102_task-procspeed_run-1_beh.csv exists
sub-103_task-nback_beh.xlsx
sub-103_task-nback_beh.csv exists
sub-103_task-procspeed_run-1_beh.txt
sub-103_task-procspeed_run-1_beh.csv exists
sub-104_task-nback_beh.xlsx
sub-104_task-nback_beh.csv exists
sub-104_task-procspeed_run-1_beh.txt
sub-104_task-procspeed_run-1_beh.csv exists
sub-105_task-nback_beh.xlsx
sub-105_task-nback_beh.csv exists
sub-105_task-procspeed_run-1_beh.txt
sub-105_task-procspeed_run-1_beh.csv exists
sub-106_task-nback_beh.xlsx
sub-106_task-nback_beh.csv exists
sub-106_task-procspeed_run-1_beh.txt
sub-106_task-procspeed_run-1_beh.csv exists
sub-107_task-nback_beh.xlsx
sub-107_task-nback_beh.csv exists
sub-107_task-procspeed_run-1_beh.txt
sub-107_task-procspeed_run-1_beh.csv exists

sub-203_task-nback_beh.csv exists
sub-203_task-procspeed_run-1_beh.txt
sub-203_task-procspeed_run-1_beh.csv exists
sub-204_task-nback_beh.xlsx
sub-204_task-nback_beh.csv exists
sub-204_task-procspeed_run-1_beh.txt
sub-204_task-procspeed_run-1_beh.csv exists
sub-205_task-nback_beh.xlsx
sub-205_task-nback_beh.csv exists
sub-205_task-procspeed_run-1_beh.txt
sub-205_task-procspeed_run-1_beh.csv exists
sub-206_task-nback_beh.xlsx
sub-206_task-nback_beh.csv exists
sub-206_task-procspeed_run-1_beh.txt
sub-206_task-procspeed_run-1_beh.csv exists
sub-207_task-nback_beh.xlsx
sub-207_task-nback_beh.csv exists
sub-207_task-procspeed_run-1_beh.txt
sub-207_task-procspeed_run-1_beh.csv exists
sub-208_task-nback_beh.xlsx
sub-208_task-nback_beh.csv exists
sub-208_task-procspeed_run-1_beh.txt
sub-208_task-procspeed_run-1_beh.csv exists
sub-209_task-nback_beh.xlsx
sub-209_task-nback_beh.csv exists
sub-209_task-procspeed_run-1_beh.txt
sub-209_task-procspeed_run-1_beh.csv exists
sub-210_task-nback_beh.xlsx

###### Output N-back trial-level data

In [5]:
pd.concat(ptb_frames).to_csv(os.path.join(derivs_dir,'nback_trial_level.csv'),index=False)

### Group, expand, trim N-back data
Group by subjects, get the sum of all columns, the count of the trial column, and the mean of the RT column.

Establish Hit % `number of Hits / number of targets` and FA % `number of FAs / number of foils`. 

Corrected Recognition `HIT% - FA%`. 

In [6]:
grouped = pd.concat(ptb_frames).groupby('sub')
nback_df = grouped.sum()
nback_df['trial'] = grouped.count()['trial']
nback_df['RT'] = grouped.mean()['RT']
nback_df['HIT%'] = nback_df['HIT'] / nback_df['CRsp']
nback_df['FA%'] = nback_df['FA'] / (nback_df['trial'] - nback_df['CRsp'])
nback_df['CoR'] = nback_df['HIT%'] - nback_df['FA%']
nback_df = nback_df[['RT','HIT%','FA%','CoR']]
nback_df.head()

,RT,HIT%,FA%,CoR
sub,,,,
101,779.888889,0.875,0.06250,0.81250
102,626.272727,0.750,0.15625,0.59375
103,618.571429,0.875,0.00000,0.87500
104,505.571429,0.875,0.00000,0.87500
105,742.913043,1.000,0.46875,0.53125


###### Output N-back subject-level data

In [7]:
nback_df.to_csv(os.path.join(derivs_dir,'nback_subject_level.csv'))

### ProcSpd data
Cleanup: Rehomogenize subject column, concatenate all frames, drop unneeded columns and rows

In [8]:
for df in ep_frames:
    df['Subject'] = df['Subject'][22]

In [9]:
procspd_df = pd.concat(ep_frames)

In [10]:
drop = ['DataFile.Basename','RandomSeed','Group','Display.RefreshRate','Clock.Information',
        'StudioVersion','RuntimeVersion','RuntimeVersionExpected','RuntimeCapabilities',
        'Experiment','SessionDate','SessionTime','SessionStartDateTimeUtc','Session',
        'TestingList','JitterList','TestingList.Cycle','TestingList.Sample',
        'BlockList','BlockList.Cycle','BlockList.Sample','Running',
        'TargetStimulus.OnsetTime','TargetStimulus.DurationError','TargetStimulus.RTTime',
        'TargetStimulus.RESP','TargetStimulus.CRESP','TargetStimulus.OnsetDelay',
        'Buffer.OnsetTime','Buffer.OnsetDelay','Buffer.DurationError','Buffer.RTTime',
        'Buffer.RESP','Buffer.CRESP'
       ]
procspd_df = procspd_df[procspd_df['Procedure']=='TrialProc'].drop(columns=drop)

###### Combine response windows for final RT

In [11]:
def calculate_rt(row):
    initial = row['TargetStimulus.RT']
    buffert = row['Buffer.RT']
    initial_duration = row['TargetStimulus.OnsetToOnsetTime']
    if initial == 0 and buffert > 0:
        rt = buffert + initial_duration
    elif initial > 0: rt = initial
    else: rt = np.nan
    return(rt)

procspd_df['RT'] = procspd_df.apply(calculate_rt,axis=1)

###### Output Procspd trial-level data

In [12]:
procspd_df.to_csv(os.path.join(derivs_dir,'procspd_trial_level.csv'),index=False)

#### Group and output ProcSpd subject-level data

In [13]:
grouped = procspd_df.groupby('Subject')
grouped.mean()[['RT']].to_csv(os.path.join(derivs_dir,'procspd_subject_level.csv'))

# Next step
## Join all subject-level data
Now that the subject-level data is cleaned & computed for ...
- Survey measures
- Comprehension
- N-back
- Processing speed

... we can combine all of that data into our final subject-level data set: [0.4.join_subject_level.ipynb](0.4.join_subject_level.ipynb)